# 1. HDB Resale Data Collection

Import dataset from excel/csv file. Data can be found in https://data.gov.sg/dataset/resale-flat-prices (if using csv/excel import, have to consolidate into a combined file). API method as per below

#### Importing the necessary libraries

In [1]:
#libraries
import urllib
import requests
import pandas as pd
import datetime
from datetime import timedelta
pd.set_option('display.max_rows', 500)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#### Calling data.gov.sg data for HDB data

- Importing them as JSON and converting them into DataFrames

In [2]:
%%time 

# #Year 1990 to 1999 (287196 records, 10 columns)
# url_1990 = 'https://data.gov.sg/api/action/datastore_search?resource_id=adbbddd3-30e2-445f-a123-29bee150a6fe&limit=1000000'
# data_1990 = requests.get(url_1990).json()
# df_1990 = pd.DataFrame.from_dict(data_1990['result']['records'])

# #Year 2000 to Feb 2012 (369651 records, 10 columns)
# url_2000 = 'https://data.gov.sg/api/action/datastore_search?resource_id=8c00bf08-9124-479e-aeca-7cc411d884c4&limit=1000000'
# data_2000 = requests.get(url_2000).json()
# df_2000 = pd.DataFrame.from_dict(data_2000['result']['records'])

# #Year Mar 2012 to Dec 2014 (52203 records, 10 columns)
# url_2012 = 'https://data.gov.sg/api/action/datastore_search?resource_id=83b2fc37-ce8c-4df4-968b-370fd818138b&limit=1000000'
# data_2012 = requests.get(url_2012).json()
# df_2012 = pd.DataFrame.from_dict(data_2012['result']['records'])

# #Year Mar 2015 to Dec 2016 (37153 records, 11 columns)
# url_2015 = 'https://data.gov.sg/api/action/datastore_search?resource_id=1b702208-44bf-4829-b620-4615ee19b57c&limit=1000000'
# data_2015 = requests.get(url_2015).json()
# df_2015 = pd.DataFrame.from_dict(data_2015['result']['records'])

#Year Jan 2017 to recent (238060 records, 11 columns)
url_2017 = 'https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit=1000000'
data_2017 = requests.get(url_2017).json()
df_2017 = pd.DataFrame.from_dict(data_2017['result']['records'])


Wall time: 16.8 s


#### Add 'remaining_lease' column to dataset that is missing. And append all 5 datasets together

In [3]:
%%time 

# #1990 -- > re-arranging and adding remaining lease 
# df_1990['remaining_lease_on_sale'] = (df_1990['lease_commence_date'].astype(int) + 99) - (pd.to_datetime(df_1990['month']).dt.year)
# df_1990 = df_1990[['_id', 'month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease_on_sale','resale_price']]

# #2000 -- > re-arranging and adding remaining lease 
# df_2000['remaining_lease_on_sale'] = (df_2000['lease_commence_date'].astype(int) + 99) - (pd.to_datetime(df_2000['month']).dt.year)
# df_2000 = df_2000[['_id', 'month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease_on_sale','resale_price']]

# #2012 -- > re-arranging columns and adding remaining lease
# df_2012['remaining_lease_on_sale'] = (df_2012['lease_commence_date'].astype(int) + 99) - (pd.to_datetime(df_2012['month']).dt.year)
# df_2012 = df_2012[['_id', 'month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease_on_sale','resale_price']]

# #2015 -- > re-arranging columns 
# df_2015 = df_2015.rename(columns={'remaining_lease': 'remaining_lease_on_sale'})
# df_2015 = df_2015[['_id', 'month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease_on_sale','resale_price']]

#2017 -- > re-arranging columns and aggregating remaining_lease column
df_2017['remaining_lease_on_sale'] = (df_2017['lease_commence_date'].astype(int) + 99) - (pd.to_datetime(df_2017['month']).dt.year)
df_2017['floor_area_sqft'] = df_2017['floor_area_sqm'].astype(float) * 10.76391042
df_2017['floor_area_sqft'] = df_2017['floor_area_sqft'].round(0)
df_2017 = df_2017[['_id', 'month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'floor_area_sqft', 'flat_model', 'lease_commence_date', 'remaining_lease_on_sale', 'resale_price']]
#df_2017 = df_2017.drop('remaining_lease', axis = 1)

df_hdb_consolidated = df_2017.copy()
df_hdb_consolidated = df_hdb_consolidated.drop('_id', axis = 1)

Wall time: 647 ms


# 2. Map data collection

#### Getting the coordinates from OneMap API, 

- Time taken to reach the nearest Hawker Centre
- Time taken to reach Raffles Place MRT Station
- Time taken to reach the nearest mall
- Time taken to reach the nearest MRT Station
- Top 30 Schools (within 2km distance)


#### Get Postal, X Coordinate, Y Coordinate, Longitude, Latitude

In [4]:
%%time
df_hdb_consolidated['street_name'] = df_hdb_consolidated['street_name'].str.replace("ST. GEORGE'S RD", "ST GEORGE'S RD") # data cleansing for API request
df_hdb_consolidated['street_name'] = df_hdb_consolidated['street_name'].str.replace("ST. GEORGE'S LANE", "ST GEORGE'S LANE") # data cleansing for API request

df_hdb_consolidated['address'] = df_hdb_consolidated['block'].astype(str) + " " + df_hdb_consolidated['street_name'].astype(str)
df_hdb_consolidated = df_hdb_consolidated[['month', 'town', 'flat_type', 'block', 'street_name', 'address', 'storey_range', 'floor_area_sqm', 'floor_area_sqft', 'flat_model', 'lease_commence_date', 'remaining_lease_on_sale', 'resale_price']]

df_hdb_consolidated.to_excel(r'output\1. HDB data (DataFrame).xlsx')

### to create for the lat and long, one time (to comment out) ###
# df_location_coordinates = pd.DataFrame(df_hdb_consolidated['address'].unique(), columns=['address'])
# url_lat_long = 'https://developers.onemap.sg/commonapi/search?returnGeom=Y&getAddrDetails=Y&searchVal='
# counter = 0

def get_LatLong(): #define as a method so that we wont keep running this
    global df_location_coordinates
    global counter

    for index in df_location_coordinates.iloc[counter:].iterrows():
        try: 
            print('Finished ' + str(counter))
            request_lat_long = url_lat_long + (index[1][0])
            response_lat_long = requests.get(request_lat_long).json()
            df_location_coordinates.loc[counter,'postal'] = pd.json_normalize(response_lat_long['results'])['POSTAL'][0] # another JSON to dataframe method
            df_location_coordinates.loc[counter,'x_coord'] = pd.json_normalize(response_lat_long['results'])['X'][0]
            df_location_coordinates.loc[counter,'y_coord'] = pd.json_normalize(response_lat_long['results'])['Y'][0]
            df_location_coordinates.loc[counter,'latitude'] = pd.json_normalize(response_lat_long['results'])['LATITUDE'][0]
            df_location_coordinates.loc[counter,'longitude'] = pd.json_normalize(response_lat_long['results'])['LONGITUDE'][0]
            counter+=1
        except KeyError:
            print('Error occurred at: ' + str(counter))
            counter+=1

#get_LatLong()

<timed exec>:1: FutureWarning: The default value of regex will change from True to False in a future version.
<timed exec>:2: FutureWarning: The default value of regex will change from True to False in a future version.


Wall time: 1min 17s


In [5]:
df_location_coordinates = pd.read_excel(r'output\2. address_lat_long_coordinates.xlsx')

#### Hawker Centre

In [ ]:
from bs4 import BeautifulSoup

infile = open(str("data/Hawker/hawker-centres-kml.kml"),"r")
contents  = infile.read()



In [ ]:
df_hdb_consolidated.head(20)

# 2. Data Preprocessing / Cleansing

# 3. Exploratory Data Analysis (EDA)